In [1]:
import pandas as pd
import datetime as dt
import ast
import os
import warnings
import zipfile

import openpyxl
from openpyxl.styles import Font, PatternFill
import openpyxl.utils



warnings.filterwarnings('ignore')
pd.options.display.max_columns=100

In [2]:
path=r'C:\Users\berid\OneDrive\Desktop\zooplus'

In [3]:
%%time
ov=pd.read_excel(os.path.join(path,'Pallets OV 2024 March.xlsx'))

CPU times: total: 375 ms
Wall time: 5.56 s


In [4]:
supplier_dict = ast.literal_eval(open(os.path.join(path,'supplier_dict.txt'),'r').read())

# Create supplier accounts

In [5]:
ov['Supplier']=ov['Supplier'].str.strip()
ov['Unloading date']=pd.to_datetime(ov['Unloading date'],errors='coerce')

In [6]:
ov.isna().sum()[ov.isna().sum()==0]

Unloading date         0
Month                  0
Reception Warehouse    0
dtype: int64

In [7]:
#clean directory

for file in os.listdir(os.path.join(path,'supplier pallet accounts/')):
    if file.endswith('xlsx'):
        file_full_name=os.path.join(path,'supplier pallet accounts',file)
        os.remove(file_full_name)

In [8]:
#add files to directory

for supplier,keyword in supplier_dict.items():
    data=ov[ov["Supplier"].str.contains(keyword,na=False,case=False,regex=True)]
    data.to_excel(os.path.join(path,'supplier pallet accounts',f"zooplus pallet acc 03-2024 {str(supplier)}.xlsx"),index=False) #month index
    print(supplier,end='\r')

In [9]:
month_dict = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

# All Pallet Accounts

# save as zipfile

zip_file_name = 'supplier data/all_pallet accounts for current month.zip'
files_directory='supplier pallet accounts/'

with zipfile.ZipFile(zip_file_name, 'w') as zip_file:
    for file_name in os.listdir(files_directory):
        file_path = os.path.join(files_directory, file_name)
        # Add files to the zip file
        zip_file.write(file_path, os.path.basename(file_path))

# Pallet Development

In [10]:
general_supplier_names=[]

for supplier_name in ov['Supplier']:
    try:
        mylist=[supplier if keyword.lower() in str(supplier_name).lower() else None for supplier,keyword in supplier_dict.items()]
        myvalue=[i for i in mylist if i][0]
        general_supplier_names.append(myvalue)
    except:
        general_supplier_names.append(None)
        
ov=ov.drop(columns='General Supplier Name',errors='ignore')       
ov.insert(ov.columns.get_loc('Supplier')+1,
         'General Supplier Name',
         general_supplier_names)


necessary_columns=['Purchase order number','Unloading date','Month','Supplier','General Supplier Name','Reception Warehouse','EUR-delivered','EUR-damaged','EUR-returned']
ov[necessary_columns].to_excel(os.path.join(path,'pallet_development.xlsx'),index=False)

# Balance Per Supplier

In [11]:
balance_dictionary={}
for file in sorted(os.listdir(os.path.join(path,'supplier pallet accounts')))[1:]:
    supplier=file.split('.xls')[0].split('2024')[-1].strip()
    df=pd.read_excel(os.path.join(path,'supplier pallet accounts',file),sheet_name='Sheet1')
    Balance=df['EUR-returned'].sum()-df['EUR-delivered'].sum()
    #df_current_month=df.query('Month=="November" and `Reception Warehouse`!="Balance 2022"') # change value for different month
    #pallets_delivered_last_month=df_current_month['EUR-delivered'].sum()
    #pallets_returned_last_month=df_current_month['EUR-returned'].sum()
    
    balance_dictionary[supplier]=[Balance]#,pallets_delivered_last_month,pallets_returned_last_month]
    print(file,end='\r')

In [12]:
balance_csv=pd.DataFrame(balance_dictionary).T
balance_csv=balance_csv.sort_index()
balance_csv.columns=['Balance']#,'Delivered','Returned']
balance_csv.to_csv(os.path.join(path,'Balances.csv'),index=True)

# Openpyxl

Add table format and a new sheet

In [13]:


def add_table_format_and_overview_worksheet(file_path):
    workbook=openpyxl.load_workbook(file_path)
    worksheet=workbook['Sheet1']

    #add table format
    max_row=worksheet.max_row
    max_col=worksheet.max_column
    table_range=f'A1:{openpyxl.utils.get_column_letter(max_col)}{max_row}'
    style=openpyxl.worksheet.table.TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,showLastColumn=False, showRowStripes=True, showColumnStripes=True)
    table=openpyxl.worksheet.table.Table(displayName='Table1',ref=table_range,tableStyleInfo=style)

    if 'Table1' in worksheet.tables:
        del worksheet.tables['Table1']
    if max_row>1:
        worksheet.add_table(table)

    #add overview worksheet
    if 'Overview' in workbook.sheetnames:
        workbook.remove(workbook['Overview'])
    workbook.create_sheet(title='Overview')

    #save workbook
    workbook.save(file_path)

In [14]:
for file in os.listdir(os.path.join(path,'supplier pallet accounts')):
    file_path=os.path.join(path,'supplier pallet accounts',file)
    try:
        add_table_format_and_overview_worksheet(file_path)
    except:
        print(file)

Add pivot table

In [15]:
import win32com.client as win32
import pythoncom


pythoncom.CoInitialize()

#launch excel application
xlApp = win32.Dispatch("Excel.Application")
xlApp.Visible = False #False

In [16]:
def add_pivot_table(file_path):

    wb = xlApp.Workbooks.Open(file_path)

    #reference worksheets
    ws_data = wb.Worksheets("Sheet1")
    ws_report = wb.Worksheets("Overview") 

    #create pivot table cache connection 
    pt_cache = wb.PivotCaches().Create(1,ws_data.Range("A1").CurrentRegion)

    #create pivot table designer/editor
    pt = pt_cache.CreatePivotTable(ws_report.Range("A3"), "myreport_summary")

    #row and column grandtotals
    pt.ColumnGrand = True
    pt.RowGrand = True

    #change report layout
    pt.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

    pt.TableStyle2 = "pivotStyleMedium9"  


    # Add row field - Reception Warehouse
    field_rows = pt.PivotFields("Reception Warehouse")
    field_rows.Orientation = 1  # 1 for xlRowField

    # Add value field (sum of EUR-delivered, EUR-damaged, EUR-returned)
    field_values = pt.PivotFields("EUR-delivered")
    field_values.Orientation = 4  # 4 for xlDataField
    field_values.Function = -4157  # -4157 for xlSum
    field_values.NumberFormat = "#,##0"  # "#,##0" for number format

    field_damaged = pt.PivotFields("EUR-damaged")
    field_damaged.Orientation = 4  # xlDataField
    field_damaged.Function = -4157  # xlSum
    field_damaged.NumberFormat = "#,##0"  # Number format

    field_returned = pt.PivotFields("EUR-returned")
    field_returned.Orientation = 4  # xlDataField
    field_returned.Function = -4157  # xlSum
    field_returned.NumberFormat = "#,##0"  # Number format

    # Add Page field for filtering - Month
    field_filter = pt.PivotFields("Month")
    field_filter.Orientation = 3  # 3 for xlPageField

    # Refresh the PivotTable to reflect the changes
    pt.RefreshTable()

    wb.save(file_path)

In [17]:
for file in os.listdir(os.path.join(path,'supplier pallet accounts')):
    file_path=os.path.join(path,'supplier pallet accounts',file)
    try:
        add_pivot_table(file_path)
    except:
        print(file)

In [18]:
xlApp.Quit() #close xlApp

Add balance cell

In [19]:
from openpyxl.styles import Font, PatternFill


def add_balance_cell(file_path):

    workbook=openpyxl.load_workbook(file_path)
    worksheet=workbook['Sheet1']
    max_row=worksheet.max_row
    max_col=worksheet.max_column

    for col in range(1,max_col+1):
        if worksheet.cell(row=1,column=col).value=="EUR-delivered":
            eur_delivered_col_index=col
        elif worksheet.cell(row=1,column=col).value=="EUR-returned":
            eur_returned_col_index=col
        elif worksheet.cell(row=1,column=col).value=="EUR-damaged":
            eur_damaged_col_index=col

    eur_delivered_sum=0
    eur_returned_sum=0
    eur_damaged_sum=0
    for row in range(2,max_row+1):
        eur_delivered_cell_value=worksheet.cell(row=row,column=eur_delivered_col_index).value
        eur_returned_cell_value=worksheet.cell(row=row,column=eur_returned_col_index).value
        eur_damaged_cell_value=worksheet.cell(row=row,column=eur_damaged_col_index).value

        try:
            eur_delivered_sum+=eur_delivered_cell_value
        except:
            pass

        try:
            eur_returned_sum+=eur_returned_cell_value
        except:
            pass

        try:
            eur_damaged_sum+=eur_damaged_cell_value
        except:
            pass
    balance=eur_returned_sum-eur_delivered_sum

    worksheet=workbook['Overview']
    balance_cell=worksheet.cell(row=25,column=1)
    if balance_cell.value:
        balance_cell.value=None
        balance_cell.font=Font()

    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')

    balance_cell.value = balance
    balance_cell.font = Font(bold=True,color='FF0000') if balance<=0 else Font(bold=True,color=None)
    balance_cell.fill=yellow_fill if balance<=0 else green_fill

    workbook.save(file_path)

In [20]:
for file in os.listdir(os.path.join(path,'supplier pallet accounts')):
    file_path=os.path.join(path,'supplier pallet accounts',file)
    try:
        add_balance_cell(file_path)
    except:
        print(file)

123